In [1]:
import numpy as np

In [2]:
class Conv2d(object):
    
    def __init__(self, input_channels, kernels, kernel_size, stride, padding):
        self.stride = stride
        self.padding = padding
        self.kernel_size = kernel.size
        self.w = np.random.randn(kernels, input_channels, kernel_size, kernel_size)
        self.b = np.random.randn(kernels)
        self.last_output = None
        self.last_input = None
        
    def zero_padding(self, X):
        if self.padding is "valid":
            return X
        elif self.padding is "same":
            pad = ((self.stride -1) * X.shape[0] - self.stride + self.kernel_size)/2
            X_padded = np.pad(X, ((0,0), (0,0), (pad, pad), (pad, pad)), mode='constant', constant_values = (0,0))
            return X_padded, pad
    
    def convolve(self, X, W, b):
        """
        Argumentos:
        X: Array numpy de entrada con dimensiones (filter_size, filter_size, n_C_prev)
        W: Array numpy con los pesos de un filtro con dimensiones (filter_size, filter_size, n_C_prev)
        b: Entero con el valor de bias de la capa actual

        Retorna:
        Z: Entero con el valor del resultado
        """

        # Multiplico elemento a elemento el valor de entrada con los pesos del filtro
        aux = X * W
        # Realizo la suma de todos los elementos
        aux = np.sum(aux)
        # Le sumo el valor del bias para obtener Z
        Z = aux + float(b)
        return Z
    
    def cov_forward(self, layer_input):
        """
        Argumentos:
        layer_input: Array numpy con los valores de entrada a la capa convolucional (batch_size, n_C_prev, n_H_prev, n_W_prev) 

        Retorna:
        Z: Array numpy con los valores de salida de la capa convolucional (batch_size, n_H, n_W, n_C)
        """
        self.last_input = layer_input

        # Obtengo las dimensiones de la entrada
        (batch_size, n_C_prev, n_H_prev, n_W_prev) = self.last_input.shape

        # Obtengo las dimensiones de los filtros
        (n_C, n_C_prev, kernel_size, kernel_size) = self.w.shape
        
        # Inicializo el volumen de salida con ceros
        Z = np.zeros([batch_size, n_C, n_H, n_W])

        # Agrego padding con ceros al volumen de entrada
        layer_input_padded, pad = zero_padding(self.last_input)
        
        # Calculo las dimensiones del volumen de salida de la capa actual
        n_H = int((n_H_prev + 2*pad - kernel_size)/self.stride + 1)
        n_W = int((n_W_prev + 2*pad - kernel_size)/self.stride + 1)

        # Comienzo iterando sobre cada ejemplo del batch
        for i in range(batch_size):

        # Itero sobre el eje vertical del volumen de salida
        for h in range(n_H):
        # Calculo las coordenadas verticales de inicio y fin de la ventana sobre la que aplicaremos el filtro
        y_start = self.stride * h
        y_end = y_start + kernel_size

        # Itero sobre el eje horizontal del volumen de salida
        for w in range(n_W):
        # Calculo las coordenadas horizontales de inicio y fin de la ventana sobre la que aplicaremos el filtro
        x_start = self.stride * w
        x_end = x_start + kernel_size

        # Extraigo la ventana para calcular la convolucion, del volumen de entrada con padding
        slice_from_input_padded = layer_input_padded[i, :, y_start:y_end, x_start:x_end]
        
        # Itero sobre la cantidad de canales del volumen de salida
        for c in range(n_C):
            
            # Obtengo el valor del filtro y bias del canal correspondiente
            kernel = self.W[c, :, :, :]
            bias = self.b[c]

            # Computo la operación de convolución para esta ventana
            self.last_output[i, c, h, w] = convolve(slice_from_input_padded, kernel, bias)
  
        return self.last_output

    def backward(self, dl_do):
        
        #dimensiones de la derivada de L con respecto de O
        (batch_size, n_C, n_H, n_W) = dl_do.shape
        
        #dimensiones de la entrada de la capa
        (batch_size, n_C_prev, n_H_prev, n_W_prev) = self.last_input.shape
        
        for c in (n_C):
            kernel = self.W[c,:,:,:]
            
            #giro 180 grados el kernel para convolucionar con la matriz de derivadas dL_dO
            conv_kernel = kernel.reshape(kernel.size)
            conv_kernel = conv_kernel[::-1]
            conv_kernel = conv_cernel.reshape(kernel.shape)
        
        
        
        dx = conv_kernel * dl_do
        
        return dx
    
    def convolve_back_df(self, dl_do, X):
        
        self.last_input = X
        df = X * dl_do
        
        
            